In [1]:
import time
from pyspark import SparkContext
sc = SparkContext('local')

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Boruvka_MST_algorithm") \
    .getOrCreate()
sc.addPyFile("graphframes-0.6.0-spark2.3-s_2.11.jar")
sc.setCheckpointDir('./')

from graphframes import *
from pyspark.sql.functions import *

In [2]:
path_to_file="2elt.edges"
edge_dataframe = sc.textFile(path_to_file)

In [3]:
edge_dataframe = edge_dataframe.map(lambda line: line.split()).map(lambda edge: \
                                                                   (edge[0], int(edge[1]),int(edge[2])))

In [4]:
begin_vex = edge_dataframe.map(lambda line: (line[0], line[0])).distinct()

In [5]:
v = spark.createDataFrame(begin_vex.collect(), ["id", "label"])
e = spark.createDataFrame(edge_dataframe.collect(), ["src", "dst", "weight"])

# Create a GraphFrame
g = GraphFrame(v, e)

In [6]:
mst = spark.createDataFrame([['', '', '']], ["src", "dst", "weight"])

In [7]:
start = time.clock()
while g.vertices.select('label').distinct().count() > 1:
    filter_df = g.find("(a)-[e]->(b)").filter("a.label != b.label").select("e.*")
    filter_df.cache()
    inter_graph = GraphFrame(g.vertices, filter_df)
    min_edges = inter_graph.triplets.groupBy('src.label').agg({'edge.weight': 'min'}).withColumnRenamed('min(edge.weight AS `weight`)', 'min_weight')

    final_edges = min_edges.join(inter_graph.triplets, (min_edges.label == inter_graph.triplets.src.label) \
                             & (min_edges.min_weight == inter_graph.triplets.edge.weight)) \
                           .select(col('src.id').alias('src'), col('dst.id').alias('dst'), col('min_weight').alias('weight'))
    final_edges.cache()
    mst = mst.union(final_edges).distinct().filter("src != ''")
    mst.cache()
    
    mst_graph = GraphFrame(g.vertices, mst)
    
    res_vertices = mst_graph.connectedComponents(algorithm='graphx',checkpointInterval=2,broadcastThreshold=10000)
    res_vertices = res_vertices.select(col('id').alias('id'), col('component').alias('label'))
    res_vertices.cache()

    g = GraphFrame(res_vertices, g.edges)
    g.cache()
elapsed = (time.clock() - start)

In [8]:
print(elapsed)

0.9436280000000001


In [9]:
mst.show()

+---+---+------+
|src|dst|weight|
+---+---+------+
|  4|  2|     9|
|  0|  9|     2|
|  3|  2|     7|
|  7|  4|    15|
|  1|  9|     5|
|  4|  8|     4|
|  1|  2|     1|
|  6|  4|    13|
|  8|  4|     4|
|  5|  0|     3|
|  9|  0|     2|
|  9|  1|     5|
|  2|  1|     1|
+---+---+------+



In [ ]:
# labelPropagation